# Projeto 2 - NLP

## Grupo:
- Laianna Lana Virginio da Silva - lvs2@cin.ufpe.br
- Lucas Natan Correia Couri - lncc2@cin.ufpe.br
- Mariama Celi Serafim de Oliveira - mcso@cin.ufpe.br

# Importando bibliotecas

In [1]:
import preprocessing as pp
import load_files as lf
import nlp_algorithms as nlp

import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer

# Carregando conjunto de dados

In [2]:
pd.set_option('display.max_colwidth', -1)
## dfSourceV2 = pd.read_csv('../Dados/v2_semeval-2022_task8_train-data_batch.csv')
train_v1 = pd.read_csv('dados/train v0.1.csv')

train_v1_enen = train_v1[(train_v1['url1_lang'] == 'en') & (train_v1['url2_lang'] == 'en')]

train_v1_enen.head()

C:\Users\mcso\AppData\Local\Temp/ipykernel_10992/2099918564.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,url1_lang,url2_lang,pair_id,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone
0,en,en,1484084337_1484110209,https://www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/local/virginia-man-arrested-in-fatal-dui-crash-in-west-virginia/2020/01/01/740fbc7a-2cbe-11ea-bffe-020c88b3f120_story.html,https://web.archive.org/web/www.washingtonpost.com/world/the_americas/haitis-leader-marks-independence-day-amid-security-concerns/2020/01/01/dc4033a4-2cc5-11ea-bffe-020c88b3f120_story.html,4.0,4.000000,1.000000,4.000000,4.000000,1.666667,2.000000
1,en,en,1484396422_1483924666,https://www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,https://web.archive.org/web/www.stlucianewsonline.com/guyana-three-injured-after-car-crashes-into-utility-pole/,https://web.archive.org/web/www.thestar.com/news/world/europe/2020/01/01/fire-kills-animals-at-zoo-in-western-germany.html,4.0,4.000000,1.000000,4.000000,3.666667,1.666667,1.333333
2,en,en,1484698254_1483758694,https://www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,https://web.archive.org/web/www.teaparty.org/trump-brings-in-2020-at-mar-a-lago-were-going-to-have-a-great-year-423052/,https://web.archive.org/web/www.timesofisrael.com/trump-says-he-does-not-expect-war-with-iran-likes-peace/,1.0,2.000000,1.000000,2.333333,2.333333,1.000000,1.333333
3,en,en,1576314516_1576455088,https://gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,https://web.archive.org/web/gadgets.ndtv.com/apps/news/zomato-uber-eats-business-acquisition-india-all-stock-deal-2167155,https://web.archive.org/web/gadgets.ndtv.com/internet/news/indian-online-food-delivery-market-to-hit-usd-8-billion-by-2020-google-bcg-report-2171043,1.0,2.333333,2.666667,1.666667,2.000000,1.666667,1.666667
4,en,en,1484036253_1483894099,https://news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,https://web.archive.org/web/news.yahoo.com/india-approves-third-moon-mission-085759387.html,https://web.archive.org/web/www.channelnewsasia.com/news/asia/india-targets-new-moon-mission-in-2020-12225344,1.0,1.250000,1.000000,1.250000,1.250000,1.000000,1.000000


In [3]:
data_path = 'dados/train v0.1/'

lista_docs = []
lista_error = []
lista_vazio = []
values = train_v1_enen[['pair_id', 'Overall']]

for index, values in values.iterrows():
    
    try:
        
        json_pair = lf.get_json_document_pair(data_path, values['pair_id'])
        text_doc1 = json_pair[0]['text']
        text_doc2 = json_pair[1]['text']
        
        if ( len(text_doc1) > 0 ) and ( len(text_doc2) > 0 ):
            lista_docs.append((values['pair_id'], text_doc1, text_doc2, values['Overall']))
        else:
            lista_vazio.append(values['pair_id'])
    
    except:
        lista_error.append(values['pair_id'])

In [4]:
df_text = pd.DataFrame(lista_docs,  columns=['pair_id', 'doc1', 'doc2', 'Overall'])
#df_text.head(1)

In [5]:
df_text.shape

(1357, 4)

# Pré-processamento dos dados

In [6]:
pre_processing_list = [
    {},
    {"basic_processing": True},
    {"basic_processing": True, "no_stopwords": True},
    {"basic_processing": True, "stemming": True},
    {"basic_processing": True, "no_stopwords": True, "stemming": True},
    {"basic_processing": True, "lema":  True},
    {"basic_processing": True, "no_stopwords": True, "lema": True}]

#df_pp = pp.pre_process_all(df_text, pre_processing_list)
df_pp = pp.pre_process_all(df_text.iloc[:10], pre_processing_list)

c:\Users\mcso\Desktop\residencia\NLP\projeto2-nlp-sidi-residencia\preprocessing.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"doc1_pipeline{index}"] = df["doc1"].apply(lambda x: pre_process(x, **param))
c:\Users\mcso\Desktop\residencia\NLP\projeto2-nlp-sidi-residencia\preprocessing.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"doc2_pipeline{index}"] = df["doc2"].apply(lambda x: pre_process(x, **param))


In [7]:
#df_pp.head(1)

# Algoritmos

In [8]:
df_results = df_pp[["pair_id", "Overall"]]
df_results.head(1)

,pair_id,Overall
0,1484084337_1484110209,4.0


## Bag of Words (CountVectorizer)

In [9]:
df_bow = nlp.apply_bow(df_pp, len(pre_processing_list))

In [10]:
df_results = df_results.join(df_bow)

In [11]:
df_results.head(5)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636
3,1576314516_1576455088,2.000000,0.678074,0.683262,0.461996,0.683147,0.482403,0.684565,0.475937
4,1484036253_1483894099,1.250000,0.790918,0.780586,0.479421,0.799176,0.540089,0.799223,0.535830


## TF-IDF

In [12]:
df_tf_idf = nlp.apply_tf_idf(df_pp, len(pre_processing_list))

In [13]:
df_results = df_results.join(df_tf_idf)

In [14]:
df_results.head(5)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,tf_idf1,tf_idf2,tf_idf3,tf_idf4,tf_idf5,tf_idf6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.204060,0.214436,0.079230,0.220277,0.074928,0.212621,0.075596
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.256470,0.253428,0.042134,0.248462,0.043084,0.249139,0.041451
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.341090,0.353646,0.234897,0.370994,0.256862,0.395499,0.280169
3,1576314516_1576455088,2.000000,0.678074,0.683262,0.461996,0.683147,0.482403,0.684565,0.475937,0.472555,0.466816,0.391316,0.485522,0.416267,0.470828,0.396233
4,1484036253_1483894099,1.250000,0.790918,0.780586,0.479421,0.799176,0.540089,0.799223,0.535830,0.570528,0.522019,0.400790,0.592051,0.487107,0.582280,0.474820


## BERT

In [15]:
'''model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')
model_name = 'base_mean'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)'''

"model = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')\nmodel_name = 'base_mean'\n\ndf_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)\n\ndf_results = df_results.join(df_bert)"

In [16]:
'''model = SentenceTransformer('sentence-transformers/bert-base-nli-max-tokens')
model_name = 'base_max'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)'''

"model = SentenceTransformer('sentence-transformers/bert-base-nli-max-tokens')\nmodel_name = 'base_max'\n\ndf_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)\n\ndf_results = df_results.join(df_bert)"

In [17]:
'''model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
model_name = 'miniLM'

df_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)

df_results = df_results.join(df_bert)'''

"model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')\nmodel_name = 'miniLM'\n\ndf_bert = nlp.apply_bert(df_pp, len(pre_processing_list), model, model_name)\n\ndf_results = df_results.join(df_bert)"

In [18]:
#df_results.head(10)

## Doc2vec

In [19]:
#Treinamento dos modelos
#nlp.train_doc2vec(df_pp, len(pre_processing_list))

In [20]:
df_doc2vec = nlp.apply_doc2vec(df_pp, len(pre_processing_list))

In [21]:
df_results = df_results.join(df_doc2vec)

In [22]:
df_results.head(10)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,...,tf_idf4,tf_idf5,tf_idf6,doc2vec0,doc2vec1,doc2vec2,doc2vec3,doc2vec4,doc2vec5,doc2vec6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.204060,...,0.074928,0.212621,0.075596,0.680492,0.693632,0.713973,0.688608,0.620387,0.753042,0.679913
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.256470,...,0.043084,0.249139,0.041451,0.439766,0.528491,0.668853,0.554909,0.560949,0.480246,0.600040
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.341090,...,0.256862,0.395499,0.280169,0.838742,0.942752,0.937498,0.879112,0.944405,0.926000,0.954795
3,1576314516_1576455088,2.000000,0.678074,0.683262,0.461996,0.683147,0.482403,0.684565,0.475937,0.472555,...,0.416267,0.470828,0.396233,0.936519,0.936305,0.973488,0.951967,0.973635,0.939475,0.960454
4,1484036253_1483894099,1.250000,0.790918,0.780586,0.479421,0.799176,0.540089,0.799223,0.535830,0.570528,...,0.487107,0.582280,0.474820,0.968844,0.974340,0.987835,0.954900,0.967951,0.968428,0.979386
5,1484189120_1484113136,2.000000,0.785585,0.777706,0.390119,0.790774,0.486256,0.780084,0.401036,0.559682,...,0.420909,0.524892,0.331954,0.961913,0.961881,0.967821,0.953431,0.961319,0.943321,0.965560
6,1484034982_1483785560,3.666667,0.485899,0.487111,0.142310,0.483234,0.145531,0.484682,0.139947,0.159859,...,0.077657,0.161401,0.073157,0.719450,0.661095,0.898157,0.537262,0.812850,0.607589,0.829112
7,1484188439_1484378177,3.333333,0.672483,0.690785,0.071027,0.678436,0.078603,0.686623,0.073578,0.300203,...,0.050321,0.314200,0.046804,0.349603,0.336715,0.497897,0.254971,0.513566,0.304435,0.469873
8,1484035244_1483938857,3.333333,0.659135,0.648809,0.222553,0.634050,0.212486,0.638550,0.211200,0.306608,...,0.114263,0.285973,0.116677,0.504499,0.636907,0.784926,0.611206,0.726026,0.611801,0.754225
9,1484396075_1483794933,2.666667,0.197386,0.199778,0.000000,0.182743,0.000000,0.182743,0.000000,0.035113,...,0.000000,0.032075,0.000000,0.020934,0.269965,0.795770,0.435752,0.769653,0.383208,0.684242


## Word2vec

In [23]:
import gensim.downloader as api

In [24]:
model = api.load('word2vec-google-news-300')

In [ ]:
df_word2vec = nlp.apply_word2vec(df_pp, len(pre_processing_list), model)

In [27]:
df_results = df_results.join(df_word2vec)

In [28]:
df_results.head(10)

,pair_id,Overall,bow0,bow1,bow2,bow3,bow4,bow5,bow6,tf_idf0,...,doc2vec4,doc2vec5,doc2vec6,word2vec_0,word2vec_1,word2vec_2,word2vec_3,word2vec_4,word2vec_5,word2vec_6
0,1484084337_1484110209,4.000000,0.554412,0.568225,0.118185,0.567762,0.118088,0.568225,0.117200,0.204060,...,0.620387,0.753042,0.679913,0.998293,0.998228,0.997522,0.997583,0.996466,0.998236,0.997508
1,1484396422_1483924666,3.666667,0.658932,0.662808,0.077949,0.652465,0.078233,0.658427,0.078285,0.256470,...,0.560949,0.480246,0.600040,0.998788,0.999105,0.998646,0.998335,0.997295,0.999010,0.998614
2,1484698254_1483758694,2.333333,0.642877,0.652648,0.395561,0.658897,0.410352,0.689574,0.465636,0.341090,...,0.944405,0.926000,0.954795,0.999040,0.999128,0.998001,0.999196,0.998049,0.999311,0.998574
3,1576314516_1576455088,2.000000,0.678074,0.683262,0.461996,0.683147,0.482403,0.684565,0.475937,0.472555,...,0.973635,0.939475,0.960454,0.998748,0.998773,0.998724,0.998679,0.998495,0.998717,0.998653
4,1484036253_1483894099,1.250000,0.790918,0.780586,0.479421,0.799176,0.540089,0.799223,0.535830,0.570528,...,0.967951,0.968428,0.979386,0.999319,0.999423,0.999252,0.999516,0.999125,0.999378,0.999234
5,1484189120_1484113136,2.000000,0.785585,0.777706,0.390119,0.790774,0.486256,0.780084,0.401036,0.559682,...,0.961319,0.943321,0.965560,0.998934,0.999009,0.998651,0.999014,0.998707,0.998986,0.998629
6,1484034982_1483785560,3.666667,0.485899,0.487111,0.142310,0.483234,0.145531,0.484682,0.139947,0.159859,...,0.812850,0.607589,0.829112,0.997923,0.998546,0.998369,0.998161,0.997229,0.998370,0.998099
7,1484188439_1484378177,3.333333,0.672483,0.690785,0.071027,0.678436,0.078603,0.686623,0.073578,0.300203,...,0.513566,0.304435,0.469873,0.997587,0.998291,0.997863,0.997753,0.997160,0.998402,0.997978
8,1484035244_1483938857,3.333333,0.659135,0.648809,0.222553,0.634050,0.212486,0.638550,0.211200,0.306608,...,0.726026,0.611801,0.754225,0.997931,0.997928,0.996805,0.997288,0.995895,0.998083,0.997171
9,1484396075_1483794933,2.666667,0.197386,0.199778,0.000000,0.182743,0.000000,0.182743,0.000000,0.035113,...,0.769653,0.383208,0.684242,0.974170,0.978725,0.959500,0.970891,0.943743,0.978179,0.957575
